# HW#3 MRET HEIN SQL Propositions
Run each SQL cell in Azure Data Studio using the **SQL** kernel connected to your database.


### Proposition 1: See the customers and their sales persons


In [ ]:
SELECT
  o.OrderID,
  o.OrderDate,
  c.CustomerName, 
  sp.PreferredName AS Salesperson,
  o.SalespersonPersonID
FROM Sales.Orders AS o
JOIN Sales.Customers      AS c  ON c.CustomerID = o.CustomerID
JOIN Application.People   AS sp ON sp.PersonID  = o.SalespersonPersonID
ORDER BY o.OrderDate, o.OrderID;

### Proposition 2: Customers who are being delivered by Delivery Van


In [ ]:
  c.CustomerID,
  c.CustomerName,
  c.DeliveryAddressLine1,
  c.DeliveryAddressLine2,
  c.DeliveryCityID,
  dm.DeliveryMethodName
FROM Sales.Customers AS c
JOIN Application.DeliveryMethods AS dm
  ON dm.DeliveryMethodID = c.DeliveryMethodID
WHERE dm.DeliveryMethodName = N'Delivery Van' 
ORDER BY c.CustomerName;

### Proposition 3: Find pairs of people with the same phone number

In [ ]:
SELECT
    p1.FullName AS Person1,
    p2.FullName AS Person2,
    p1.PhoneNumber
FROM Application.People AS p1
JOIN Application.People AS p2
    ON p1.PhoneNumber = p2.PhoneNumber
WHERE p1.PersonID < p2.PersonID;

### Proposition 4: Top 10 stock items in warehouse by total sales amount (revenue)

In [ ]:
SELECT TOP (10)
       si.StockItemID,
       si.StockItemName,
       SUM(il.ExtendedPrice) AS TotalSalesAmount,
       SUM(il.Quantity)      AS UnitsSold
FROM Sales.InvoiceLines AS il
JOIN Sales.Invoices     AS i  
	ON i.InvoiceID   = il.InvoiceID
JOIN Warehouse.StockItems AS si 
	ON si.StockItemID = il.StockItemID
GROUP BY si.StockItemID, si.StockItemName
ORDER BY TotalSalesAmount DESC, si.StockItemName;

### Proposition 5: Find customers who had a shipment on a certain day and then another shipment the very next day. 


In [ ]:
SELECT
    o1.CustomerID,
    o1.OrderID AS ORDER1ID,
    o1.OrderDate AS FirstOrderDate,
    o2.OrderID AS Order2ID,
    o2.OrderDate AS SecondOrderDate
FROM Sales.Orders AS o1
JOIN Sales.Orders AS o2
    ON o1.CustomerID = o2.CustomerID
WHERE DATEDIFF(day, o1.OrderDate, o2.OrderDate) = 1;

### Proposition 6: The query below will show all suppliers, their corresponding cities, and the total number of purchase orders they have received. It's then ordered to show the busiest suppliers first.


In [ ]:
SELECT
    s.SupplierName,
    c.CityName,
    COUNT(po.PurchaseOrderID) AS NumberOfPurchaseOrders
FROM Purchasing.Suppliers AS s
JOIN Purchasing.PurchaseOrders AS po 
   ON s.SupplierID = po.SupplierID
JOIN Application.Cities AS c 
   ON s.DeliveryCityID = c.CityID
GROUP BY
    s.SupplierName,
    c.CityName
ORDER BY
    NumberOfPurchaseOrders DESC;


### Proposition 7: Top debtors (total open balance per customer) for accounts receivables 


In [ ]:
SELECT
  c.CustomerID,
  c.CustomerName,
  SUM(ct.OutstandingBalance) AS TotalOutstanding
FROM Sales.CustomerTransactions AS ct
JOIN Sales.Customers AS c ON c.CustomerID = ct.CustomerID
WHERE ct.OutstandingBalance > 0
GROUP BY c.CustomerID, c.CustomerName
ORDER BY TotalOutstanding DESC, c.CustomerName;

### Proposition 8: Calculate credit utilization (open balance ÷ credit limit)


In [ ]:
SELECT
  c.CustomerID,
  c.CustomerName,
  c.CreditLimit,
  SUM(ct.OutstandingBalance) AS TotalOutstanding,
  CASE WHEN c.CreditLimit IS NULL OR c.CreditLimit = 0
       THEN NULL
       ELSE SUM(ct.OutstandingBalance) / c.CreditLimit END AS UtilizationRatio
FROM Sales.Customers AS c
LEFT JOIN Sales.CustomerTransactions AS ct
  ON ct.CustomerID = c.CustomerID AND ct.OutstandingBalance > 0
GROUP BY c.CustomerID, c.CustomerName, c.CreditLimit
ORDER BY UtilizationRatio DESC, TotalOutstanding DESC, c.CustomerName;

### Proposition 9: Top items by total tax collected


In [ ]:
SELECT 
       si.StockItemID,
       si.StockItemName,
       SUM(il.TaxAmount)   AS TotalTaxCollected,
       SUM(il.ExtendedPrice)   AS RevenueExclTax,
       AVG(il.TaxRate)       AS AvgTaxRate
FROM Sales.InvoiceLines AS il
JOIN Sales.Invoices     AS i  ON i.InvoiceID    = il.InvoiceID
JOIN Warehouse.StockItems AS si ON si.StockItemID = il.StockItemID
GROUP BY si.StockItemID, si.StockItemName
ORDER BY TotalTaxCollected DESC, si.StockItemName;

### Proposition 10: Planned average time (Order → ExpectedDeliveryDate) by city



In [ ]:
SELECT
  ci.CityName,
  AVG(DATEDIFF(minute, o.OrderDate, o.ExpectedDeliveryDate) / 60.0) AS Avg_Hours_Planned,
  COUNT(*) AS Orders
FROM Sales.Orders AS o
JOIN Sales.Customers AS c  ON c.CustomerID = o.CustomerID
JOIN Application.Cities AS ci ON ci.CityID = c.DeliveryCityID
WHERE o.OrderDate IS NOT NULL
  AND o.ExpectedDeliveryDate IS NOT NULL
  AND o.ExpectedDeliveryDate >= o.OrderDate
GROUP BY ci.CityName
ORDER BY Avg_Hours_Planned ASC, ci.CityName;